## Feature Request =)

It would be cool to see some statistics such as elevation / inclination / average speed along the track (on a map). 


This demo notebook, shows both elevation and inclination as two seperate layers on the map. You can switch on/off both layers in the LayerControl widget (top-right).

Data used: https://www.komoot.com/tour/41456388


In [1]:
import folium
import datetime
import gpxpy
import matplotlib.pyplot as plt
import pandas as pd
import srtm
from statistics import mean
import numpy as np

%matplotlib inline

/Users/LucasvanWalstijn/miniconda3/envs/track/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/LucasvanWalstijn/miniconda3/envs/track/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
GPX_FILE = './data/2018-08-05_41326464_zichower-runde_export.gpx'

In [3]:
gpx = gpxpy.parse(open(GPX_FILE))
elevation_data = srtm.get_data()
elevation_data.add_elevations(gpx)

In [4]:
def plot_track(zoom=11):
    m = folium.Map()
    
    points = [p for p in gpx.get_points_data()]
    mean_lat = mean(p.point.latitude for p in points)
    mean_lng = mean(p.point.longitude for p in points)
    
    m.location = [mean_lat, mean_lng]
    m.zoom_start = zoom
    
    pts = [(p.point.latitude, p.point.longitude) for p in points]
    elevation_pts = [p.point.elevation for p in points]
        
    dist_from_start = [p.distance_from_start for p in points]
    dist_travelled = np.array(dist_from_start + [dist_from_start[-1]]) - np.array([dist_from_start[0]] + dist_from_start)
    elevation_increase = np.array(elevation_pts + [elevation_pts[-1]]) - np.array([elevation_pts[0]] + elevation_pts)
    
    rel_elevation_increase = ((elevation_increase / dist_travelled)) * 100
    rel_elevation_increase[0] = 0
    rel_elevation_increase[-1] = 0
    
    #make a cutoff, otherwise because of huge outliers most values will turn out average (->yellow)
    cutoff = 10
    rel_elevation_increase = [p if np.abs(p) < cutoff else np.sign(p) * cutoff for p in rel_elevation_increase]
    
    folium.features.ColorLine(pts, colors=elevation_pts, weight=4, opacity=1, name ="elevation (course)").add_to(m)
    folium.features.ColorLine(pts, colors=rel_elevation_increase, weight=4, opacity=1, name="inclination (fine)").add_to(m)
    
    folium.LayerControl().add_to(m)
    
    display(m)

In [5]:
plot_track()

/Users/LucasvanWalstijn/miniconda3/envs/track/lib/python3.5/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
